In [1]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from numba import jit, float64, int64

In [2]:
@jit(nopython=True)
def Race(v, b, ndt=0, dt=0.001):
    x = np.array([0.0, 0.0])
    rt = 0
    ch = -1
    stop = False
    while not stop:
        x[0] += v[0]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        x[1] += v[1]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        rt += dt
        if x[0]>= b and x[1]<b:
            ch = 0.0
            stop = True
        elif x[1]>= b and x[0]<b:
            ch = 1.0
            stop = True
        elif x[0]>= b and x[0]>=b:
            rt = 0
            x = np.array([0.0, 0.0])
    return rt+ndt, ch

In [3]:
data_df = pd.read_csv('../../_00_data/Yang2024_prep.csv', index_col=0)

min_rt = data_df.groupby(by='sbj')['rt'].min()

tr_df, ts_df = data_df.copy(), data_df.copy()

obs_train = np.asarray(data_df[['rt', 'Right', 'sbj']])
X_train = np.asarray(data_df[["o1_a1_val", "o1_a2_val", "o2_a1_val", "o2_a2_val",
                              "o1_a1_gaze", "o1_a2_gaze", "o2_a1_gaze", "o2_a2_gaze"]])

obs_test = np.asarray(data_df[['rt', 'Right', 'sbj']])
X_test = np.asarray(data_df[["o1_a1_val", "o1_a2_val", "o2_a1_val", "o2_a2_val",
                             "o1_a1_gaze", "o1_a2_gaze", "o2_a1_gaze", "o2_a2_gaze"]])

print(X_train.shape, X_test.shape)

(9731, 8) (9731, 8)


In [4]:
def fpt(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(b/(2*np.pi*t**3)**(1/2) * keras.backend.exp(-1/(2*t) * (v*t-b)**2)) + tf.cast(tf.math.less_equal(t, 0), tf.float32)*0.65**11

def phi(x):
    return 0.5*tf.math.erf(x/tf.sqrt(2.0)) + 0.5

def cdf(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(phi((v*t-b)/tf.sqrt(t)) + tf.exp(2*v*b)*phi((-v*t-b)/tf.sqrt(t))) + (tf.cast(tf.math.less_equal(t, 0), tf.float32)*0)

def G2(data, pred):

    sbj_indices = tf.cast(data[:, 2], tf.int32)

    # Gather values directly using TensorFlow operations
    gathered_min_rt = tf.gather(min_rt, sbj_indices)
    sigmoid_weights = tf.gather(model.layers[-1].weights[-1], sbj_indices)
    ndt = gathered_min_rt * tf.math.sigmoid(sigmoid_weights)

    thr_weights = tf.gather(model.layers[-1].weights[-2], sbj_indices)
    thr = tf.math.log(1 + tf.exp(thr_weights))

    scl_weights = tf.gather(model.layers[-1].weights[-3], sbj_indices)
    scl = tf.math.log(1 + tf.exp(scl_weights))

    ll1 = (1 - data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 0]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 1]))
    ll2 = (data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 1]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 0]))

    return -2*tf.reduce_sum(tf.math.log(tf.maximum(ll1 + ll2, 3e-18)), axis=-1)

In [5]:
class Out_layer(keras.layers.Layer):
    def __init__(self, units=2, input_dim=32, n_sbj_prms=10):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(units,), initializer="uniform", trainable=True)
        
        self.scaling = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.threshold = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.non_decision_time = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)

    def call(self, inputs):
        return tf.math.log(1 + tf.exp(tf.matmul(inputs, self.w) + self.b))
    
def make_model(layers_neuron, input_shape, output_shape, activation, n_sbj_prms):
    keras.backend.clear_session()

    model = keras.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    
    for l in layers_neuron:
        model.add(layers.Dense(l, activation=activation))
    
    model.add(Out_layer(units=output_shape, input_dim=layers_neuron[-1], n_sbj_prms=n_sbj_prms))

    print(model.summary())
    
    return model

In [6]:
layers_neuron = [10, 10, 8, 5]
input_shape = 8
output_shape = 2
activation = 'tanh'
n_sbj = data_df.sbj.nunique()

model = make_model(layers_neuron, input_shape, output_shape, activation, n_sbj)

model.build()
model.compile(optimizer="RMSprop", loss=G2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                90        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
 dense_3 (Dense)             (None, 5)                 45        
                                                                 
 out_layer (Out_layer)       (None, 2)                 96        
                                                                 
Total params: 429 (1.68 KB)
Trainable params: 429 (1.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [7]:
sbj_indices = tf.cast(obs_train[:, 2], tf.int32)
obs_train = tf.cast(obs_train, tf.float32)
X_train = tf.convert_to_tensor(X_train)
X_train = tf.cast(X_train, tf.float32)

min_rt = tf.convert_to_tensor(min_rt)
min_rt = tf.cast(min_rt, tf.float32)

In [8]:
history = model.fit(X_train, 
                    obs_train, 
                    shuffle=False, 
                    batch_size=X_train.shape[0], 
                    epochs=5000)

Epoch 1/5000
1/1 [==============================] - 1s 591ms/step - loss: 46621.2578
Epoch 2/5000
1/1 [==============================] - 0s 3ms/step - loss: 46422.8594
Epoch 3/5000
1/1 [==============================] - 0s 3ms/step - loss: 46282.3555
Epoch 4/5000
1/1 [==============================] - 0s 3ms/step - loss: 46165.1172
Epoch 5/5000
1/1 [==============================] - 0s 3ms/step - loss: 46060.8164
Epoch 6/5000
1/1 [==============================] - 0s 3ms/step - loss: 45965.0664
Epoch 7/5000
1/1 [==============================] - 0s 4ms/step - loss: 45875.5000
Epoch 8/5000
1/1 [==============================] - 0s 3ms/step - loss: 45790.6484
Epoch 9/5000
1/1 [==============================] - 0s 4ms/step - loss: 45709.5234
Epoch 10/5000
1/1 [==============================] - 0s 3ms/step - loss: 45631.4141
Epoch 11/5000
1/1 [==============================] - 0s 4ms/step - loss: 45555.8047
Epoch 12/5000
1/1 [==============================] - 0s 3ms/step - loss: 45482.3125

1/1 [==============================] - 0s 4ms/step - loss: 41162.6875
Epoch 99/5000
1/1 [==============================] - 0s 4ms/step - loss: 41125.7266
Epoch 100/5000
1/1 [==============================] - 0s 4ms/step - loss: 41089.0000
Epoch 101/5000
1/1 [==============================] - 0s 4ms/step - loss: 41052.4961
Epoch 102/5000
1/1 [==============================] - 0s 5ms/step - loss: 41016.2227
Epoch 103/5000
1/1 [==============================] - 0s 4ms/step - loss: 40980.1719
Epoch 104/5000
1/1 [==============================] - 0s 4ms/step - loss: 40944.3438
Epoch 105/5000
1/1 [==============================] - 0s 4ms/step - loss: 40908.7305
Epoch 106/5000
1/1 [==============================] - 0s 5ms/step - loss: 40873.3359
Epoch 107/5000
1/1 [==============================] - 0s 4ms/step - loss: 40838.1523
Epoch 108/5000
1/1 [==============================] - 0s 4ms/step - loss: 40803.1875
Epoch 109/5000
1/1 [==============================] - 0s 4ms/step - loss: 40768.4

1/1 [==============================] - 0s 4ms/step - loss: 38327.2891
Epoch 195/5000
1/1 [==============================] - 0s 4ms/step - loss: 38300.4844
Epoch 196/5000
1/1 [==============================] - 0s 4ms/step - loss: 38273.6641
Epoch 197/5000
1/1 [==============================] - 0s 4ms/step - loss: 38246.8164
Epoch 198/5000
1/1 [==============================] - 0s 4ms/step - loss: 38219.9453
Epoch 199/5000
1/1 [==============================] - 0s 4ms/step - loss: 38193.0273
Epoch 200/5000
1/1 [==============================] - 0s 4ms/step - loss: 38166.0586
Epoch 201/5000
1/1 [==============================] - 0s 4ms/step - loss: 38139.0312
Epoch 202/5000
1/1 [==============================] - 0s 4ms/step - loss: 38111.9258
Epoch 203/5000
1/1 [==============================] - 0s 4ms/step - loss: 38084.7266
Epoch 204/5000
1/1 [==============================] - 0s 4ms/step - loss: 38057.4297
Epoch 205/5000
1/1 [==============================] - 0s 4ms/step - loss: 38030.

1/1 [==============================] - 0s 4ms/step - loss: 35462.7969
Epoch 291/5000
1/1 [==============================] - 0s 4ms/step - loss: 35431.0547
Epoch 292/5000
1/1 [==============================] - 0s 4ms/step - loss: 35399.2422
Epoch 293/5000
1/1 [==============================] - 0s 4ms/step - loss: 35367.3594
Epoch 294/5000
1/1 [==============================] - 0s 6ms/step - loss: 35335.4062
Epoch 295/5000
1/1 [==============================] - 0s 5ms/step - loss: 35303.3828
Epoch 296/5000
1/1 [==============================] - 0s 4ms/step - loss: 35271.2852
Epoch 297/5000
1/1 [==============================] - 0s 4ms/step - loss: 35239.0977
Epoch 298/5000
1/1 [==============================] - 0s 4ms/step - loss: 35206.8398
Epoch 299/5000
1/1 [==============================] - 0s 5ms/step - loss: 35174.5000
Epoch 300/5000
1/1 [==============================] - 0s 4ms/step - loss: 35142.0781
Epoch 301/5000
1/1 [==============================] - 0s 4ms/step - loss: 35109.

1/1 [==============================] - 0s 5ms/step - loss: 32405.1934
Epoch 387/5000
1/1 [==============================] - 0s 4ms/step - loss: 32375.7617
Epoch 388/5000
1/1 [==============================] - 0s 4ms/step - loss: 32346.3711
Epoch 389/5000
1/1 [==============================] - 0s 4ms/step - loss: 32317.0273
Epoch 390/5000
1/1 [==============================] - 0s 4ms/step - loss: 32287.7148
Epoch 391/5000
1/1 [==============================] - 0s 4ms/step - loss: 32258.4492
Epoch 392/5000
1/1 [==============================] - 0s 5ms/step - loss: 32229.2148
Epoch 393/5000
1/1 [==============================] - 0s 4ms/step - loss: 32200.0312
Epoch 394/5000
1/1 [==============================] - 0s 4ms/step - loss: 32170.8711
Epoch 395/5000
1/1 [==============================] - 0s 4ms/step - loss: 32141.7598
Epoch 396/5000
1/1 [==============================] - 0s 4ms/step - loss: 32112.6699
Epoch 397/5000
1/1 [==============================] - 0s 4ms/step - loss: 32083.

...

1/1 [==============================] - 0s 4ms/step - loss: 16076.2021
Epoch 4573/5000
1/1 [==============================] - 0s 4ms/step - loss: 15926.1426
Epoch 4574/5000
1/1 [==============================] - 0s 4ms/step - loss: 15923.5742
Epoch 4575/5000
1/1 [==============================] - 0s 4ms/step - loss: 15870.3457
Epoch 4576/5000
1/1 [==============================] - 0s 4ms/step - loss: 15914.0146
Epoch 4577/5000
1/1 [==============================] - 0s 4ms/step - loss: 15970.7998
Epoch 4578/5000
1/1 [==============================] - 0s 4ms/step - loss: 15922.1074
Epoch 4579/5000
1/1 [==============================] - 0s 4ms/step - loss: 15854.8291
Epoch 4580/5000
1/1 [==============================] - 0s 4ms/step - loss: 15924.5645
Epoch 4581/5000
1/1 [==============================] - 0s 4ms/step - loss: 15843.5166
Epoch 4582/5000
1/1 [==============================] - 0s 4ms/step - loss: 15881.6025
Epoch 4583/5000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 5ms/step - loss: 16388.0762
Epoch 4668/5000
1/1 [==============================] - 0s 5ms/step - loss: 15970.5850
Epoch 4669/5000
1/1 [==============================] - 0s 4ms/step - loss: 15882.2139
Epoch 4670/5000
1/1 [==============================] - 0s 4ms/step - loss: 15789.8672
Epoch 4671/5000
1/1 [==============================] - 0s 5ms/step - loss: 15652.5869
Epoch 4672/5000
1/1 [==============================] - 0s 4ms/step - loss: 15669.7354
Epoch 4673/5000
1/1 [==============================] - 0s 5ms/step - loss: 15661.9609
Epoch 4674/5000
1/1 [==============================] - 0s 4ms/step - loss: 15748.5303
Epoch 4675/5000
1/1 [==============================] - 0s 5ms/step - loss: 15850.5596
Epoch 4676/5000
1/1 [==============================] - 0s 4ms/step - loss: 15938.7197
Epoch 4677/5000
1/1 [==============================] - 0s 5ms/step - loss: 15861.5439
Epoch 4678/5000
1/1 [==============================] - 0s 5ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 16002.0781
Epoch 4763/5000
1/1 [==============================] - 0s 4ms/step - loss: 15832.5186
Epoch 4764/5000
1/1 [==============================] - 0s 4ms/step - loss: 15963.1133
Epoch 4765/5000
1/1 [==============================] - 0s 4ms/step - loss: 15978.1680
Epoch 4766/5000
1/1 [==============================] - 0s 4ms/step - loss: 15927.9121
Epoch 4767/5000
1/1 [==============================] - 0s 4ms/step - loss: 15842.2178
Epoch 4768/5000
1/1 [==============================] - 0s 4ms/step - loss: 15938.0703
Epoch 4769/5000
1/1 [==============================] - 0s 4ms/step - loss: 15782.4434
Epoch 4770/5000
1/1 [==============================] - 0s 4ms/step - loss: 15818.9971
Epoch 4771/5000
1/1 [==============================] - 0s 4ms/step - loss: 15763.5186
Epoch 4772/5000
1/1 [==============================] - 0s 4ms/step - loss: 15899.5293
Epoch 4773/5000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 15839.4297
Epoch 4858/5000
1/1 [==============================] - 0s 5ms/step - loss: 15931.7041
Epoch 4859/5000
1/1 [==============================] - 0s 4ms/step - loss: 15776.1133
Epoch 4860/5000
1/1 [==============================] - 0s 4ms/step - loss: 15858.0830
Epoch 4861/5000
1/1 [==============================] - 0s 4ms/step - loss: 15769.6104
Epoch 4862/5000
1/1 [==============================] - 0s 4ms/step - loss: 15851.9072
Epoch 4863/5000
1/1 [==============================] - 0s 4ms/step - loss: 15841.4375
Epoch 4864/5000
1/1 [==============================] - 0s 4ms/step - loss: 15860.4375
Epoch 4865/5000
1/1 [==============================] - 0s 4ms/step - loss: 15840.2627
Epoch 4866/5000
1/1 [==============================] - 0s 4ms/step - loss: 15845.0508
Epoch 4867/5000
1/1 [==============================] - 0s 4ms/step - loss: 15834.3672
Epoch 4868/5000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 5ms/step - loss: 15789.3232
Epoch 4953/5000
1/1 [==============================] - 0s 5ms/step - loss: 15887.0723
Epoch 4954/5000
1/1 [==============================] - 0s 5ms/step - loss: 15795.5840
Epoch 4955/5000
1/1 [==============================] - 0s 5ms/step - loss: 15927.7227
Epoch 4956/5000
1/1 [==============================] - 0s 6ms/step - loss: 15816.7305
Epoch 4957/5000
1/1 [==============================] - 0s 5ms/step - loss: 15944.0713
Epoch 4958/5000
1/1 [==============================] - 0s 5ms/step - loss: 15796.3115
Epoch 4959/5000
1/1 [==============================] - 0s 5ms/step - loss: 15835.8945
Epoch 4960/5000
1/1 [==============================] - 0s 4ms/step - loss: 15726.9580
Epoch 4961/5000
1/1 [==============================] - 0s 5ms/step - loss: 15809.4648
Epoch 4962/5000
1/1 [==============================] - 0s 5ms/step - loss: 15780.5117
Epoch 4963/5000
1/1 [==============================] - 0s 5ms/step - l

In [9]:
pred_v = model.predict(X_test)
threshold = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-2])))
ndt = np.asanyarray(np.asanyarray(min_rt)*tf.math.sigmoid(model.layers[-1].weights[-1]))

np.save('input.npy', X_test)
np.save('output.npy', pred_v)

305/305 [==============================] - 0s 306us/step


In [10]:
pred_bhv = {'sbj': [],
            'rt': [],
            'ch': [],
            'value_diff': [],
            'gaze_diff': []}

scaling = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-3])))
scaling = np.asanyarray([scaling[i] for i in ts_df['sbj']])

for n in range(20):
    for s in range(ts_df.shape[0]):
        rt, ch = Race(scaling[s]*pred_v[s,:], threshold[ts_df.sbj[s]], ndt[ts_df.sbj[s]])
        pred_bhv['sbj'].append(ts_df.sbj[s])
        pred_bhv['rt'].append(rt)
        pred_bhv['ch'].append(ch)
        pred_bhv['value_diff'].append(ts_df['Right Value - Left Value'][s])
        pred_bhv['gaze_diff'].append(ts_df['Dwell Time Advantage (R - L)'][s])
        
pred_bhv = pd.DataFrame(pred_bhv)
pred_bhv.to_csv('Yang_complete.csv')